In [ ]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

: 

In [2]:
import os
import warnings
import pandas as pd
import torch
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, default_data_collator
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from torch.utils.data import  DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
import numpy as np
from torch.optim import AdamW


[2025-06-12 12:32:47,080] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/sohampoddar/anaconda3/envs/llm/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/sohampoddar/anaconda3/envs/llm/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda-12/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/sohampoddar/anaconda3/envs/llm/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda-12/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/sohampoddar/anaconda3/envs/llm/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda-12/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/sohampoddar/anaconda3/envs/llm/compiler_compat/ld: /usr/local/cuda-12/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/home/sohampoddar/anaconda3/envs/llm/compiler_compat/ld: /usr/local/cuda-12/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/home/sohampodda

In [3]:
from torch.utils.data import Dataset

class TweetDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]  # This is already a list of binary values
        
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.FloatTensor(label)  # Convert to float for BCEWithLogitsLoss
        }

In [4]:
train_data_path = "/home/sohampoddar/HDD2/utsav/Dataset/train.csv"
val_data_path = "/home/sohampoddar/HDD2/utsav/Dataset/test.csv"
train_data = pd.read_csv(train_data_path)
val_data = pd.read_csv(val_data_path)

In [5]:
label_list = ['rushed','side-effect','ineffective','mandatory','pharma','ingredients',
              'country','conspiracy','political','unnecessary','none']
mlb = MultiLabelBinarizer(classes=label_list)

# Convert space-separated label string to list
train_data['labels'] = train_data['labels'].apply(lambda x: x.split(' '))
val_data['labels'] = val_data['labels'].apply(lambda x: x.split(' '))
# Transform to binary format (2D array)
binary_labels = mlb.fit_transform(train_data['labels'])
binary_labels_val = mlb.fit_transform(val_data['labels'])

# Store as list of 11 binary values per tweet
train_data['labels'] = binary_labels.tolist()
val_data['labels'] = binary_labels_val.tolist()

# Extract final lists
train_texts = train_data['text'].tolist()
val_texts = val_data['text'].tolist()
train_labels = train_data['labels'].tolist()
val_labels = val_data['labels'].tolist()


/home/sohampoddar/anaconda3/envs/llm/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['religious'] will be ignored
  warnings.warn(


In [7]:
tokenizer = AutoTokenizer.from_pretrained('roberta-large')
max_len = 512
train_dataset = TweetDataset(train_texts, train_labels, tokenizer, max_len)
test_dataset = TweetDataset(val_texts, val_labels, tokenizer, max_len)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

model = RobertaForSequenceClassification.from_pretrained(
    'roberta-large',
    num_labels=len(label_list),
    problem_type="multi_label_classification"
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

optimizer = AdamW(model.parameters(), lr=1e-5)

loss_fn = torch.nn.BCEWithLogitsLoss()

epochs = 6

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} complete. Avg Loss: {avg_loss:.4f}")

# --- Step 7: Evaluation ---
model.eval()
all_preds, all_labels = [], []

val_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        probs = torch.sigmoid(outputs.logits)
        preds = (probs > 0.5).int()
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1 complete. Avg Loss: 0.2543
Epoch 2 complete. Avg Loss: 0.1545
Epoch 3 complete. Avg Loss: 0.1214
Epoch 4 complete. Avg Loss: 0.0957
Epoch 5 complete. Avg Loss: 0.0742
Epoch 6 complete. Avg Loss: 0.0590


In [8]:
# save the model 
model.save_pretrained('roberta_large_model')
# save the tokenizer 
tokenizer.save_pretrained('roberta_large_tokenizer')

('roberta_large_tokenizer/tokenizer_config.json',
 'roberta_large_tokenizer/special_tokens_map.json',
 'roberta_large_tokenizer/vocab.json',
 'roberta_large_tokenizer/merges.txt',
 'roberta_large_tokenizer/added_tokens.json',
 'roberta_large_tokenizer/tokenizer.json')

In [9]:
# load the model 
model = RobertaForSequenceClassification.from_pretrained('roberta_large_model')
model.to(device)
tokenizer = RobertaTokenizer.from_pretrained('roberta_large_tokenizer')

# test the model 


model.eval()
all_preds, all_labels = [], []

max_len = 512

test_dataset = TweetDataset(val_texts, val_labels, tokenizer, max_len)


val_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        probs = torch.sigmoid(outputs.logits)
        preds = (probs > 0.5).int()
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())


In [10]:
print(classification_report(all_labels, all_preds, target_names=label_list, digits=4))

              precision    recall  f1-score   support

      rushed     0.7692    0.7458    0.7573       295
 side-effect     0.8361    0.8701    0.8527       762
 ineffective     0.6611    0.8293    0.7357       334
   mandatory     0.7452    0.7452    0.7452       157
      pharma     0.7178    0.6784    0.6976       255
 ingredients     0.7463    0.5747    0.6494        87
     country     0.8125    0.6500    0.7222        40
  conspiracy     0.6024    0.5155    0.5556        97
   political     0.6667    0.5600    0.6087       125
 unnecessary     0.6268    0.6138    0.6202       145
        none     0.6709    0.4206    0.5171       126

   micro avg     0.7438    0.7379    0.7408      2423
   macro avg     0.7141    0.6549    0.6783      2423
weighted avg     0.7427    0.7379    0.7364      2423
 samples avg     0.7508    0.7570    0.7379      2423



/home/sohampoddar/anaconda3/envs/llm/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sohampoddar/anaconda3/envs/llm/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sohampoddar/anaconda3/envs/llm/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capi

In [11]:
from sklearn.metrics import classification_report, f1_score, accuracy_score
# Convert lists to NumPy arrays for metric functions
y_true = np.array(all_labels)
y_pred = np.array(all_preds)

# F1 Scores
f1_micro = f1_score(y_true, y_pred, average='micro')
f1_macro = f1_score(y_true, y_pred, average='macro')

# Accuracy (strict = all labels correct)
strict_accuracy = accuracy_score(y_true, y_pred)

print(f"F1 Micro: {f1_micro:.4f}")
print(f"F1 Macro: {f1_macro:.4f}")
print(f"Strict Accuracy: {strict_accuracy:.4f}")

F1 Micro: 0.7408
F1 Macro: 0.6783
Strict Accuracy: 0.5964
